<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# Проект «Анализ вакансий из HeadHunter»
   

In [1]:
#%pip install psycopg2

In [2]:
import pandas as pd
import psycopg2

In [56]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [57]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

In [58]:
cur = connection.cursor()

## 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в базе (вакансии находятся в таблице `vacancies`).

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT count(*) FROM public.vacancies
'''

In [7]:
# результат запроса
cur.execute(query_3_1)
cur.fetchone()

(49197,)

2. Напишите запрос, который посчитает количество работодателей (таблица `employers`).

In [ ]:
# текст запроса
query_3_2 = f'''
SELECT count(*) FROM public.employers
'''

In [9]:
# результат запроса
cur.execute(query_3_2)
cur.fetchone()

(23501,)

3. Посчитайте с помощью запроса количество регионов (таблица `areas`).

In [ ]:
# текст запроса
query_3_3 = f'''
SELECT count(*) FROM public.areas
'''

In [11]:
# результат запроса
cur.execute(query_3_3)
cur.fetchone()

(1362,)

4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица `industries`).

In [ ]:
# текст запроса
query_3_4 = f'''
SELECT count(*) FROM public.industries
'''

In [13]:
# результат запроса
cur.execute(query_3_4)
cur.fetchone()

(294,)

***

# выводы по предварительному анализу данных
1. Количество вакансий в базе (таблица `vacancies`):
   - Всего: **49 197** вакансий

2. Количество работодателей (таблица `employers`):
   - Всего: **3 501** работодатель

3. Количество регионов (таблица `areas`):
   - Всего: **1 362** региона

4. Количество сфер деятельности (таблица `industries`):
   - Всего: **294** сферы деятельности

### Выводы по предварительному анализу данных

Итого на **49 197** вакансий приходится **23 501** работодатель, распределённые по **1 362** регионам. Всего в базе данных представлены вакансии по **294** различным сферам деятельности.

## 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (`cnt`) вакансий в каждом регионе (`area`).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
# текст запроса
# текст запроса
query_4_1 = f'''
SELECT 
    a.name AS area,
    count(v.id) AS cnt
FROM public.areas a
LEFT JOIN public.vacancies v ON a.id = v.area_id
GROUP BY a.id, a.name
ORDER BY cnt DESC
'''

In [15]:
# результат запроса
cur.execute(query_4_1)
results = cur.fetchmany(5)
for row in results:
    print(row)

('Москва', 5333)
('Санкт-Петербург', 2851)
('Минск', 2112)
('Новосибирск', 2006)
('Алматы', 1892)


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
# текст запроса
query_4_2 = f'''
SELECT count(*) 
FROM public.vacancies 
WHERE salary_from IS NOT NULL 
   OR salary_to IS NOT NULL
'''

In [17]:
# результат запроса
cur.execute(query_4_2)
cur.fetchone()

(24073,)

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до **целого числа**.

In [18]:
# текст запроса
query_4_3 = f'''
SELECT 
    ROUND(AVG(salary_from)) AS avg_salary_from,
    ROUND(AVG(salary_to)) AS avg_salary_to
FROM public.vacancies 
WHERE salary_from IS NOT NULL 
   OR salary_to IS NOT NULL
'''

In [19]:
# результат запроса
cur.execute(query_4_3)
cur.fetchone()

(Decimal('71065'), Decimal('110537'))

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (`schedule`) и типа трудоустройства (`employment`), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
# текст запроса
query_4_4 = f'''
SELECT 
    schedule,
    employment,
    count(*) AS vacancy_count
FROM public.vacancies
WHERE schedule IS NOT NULL 
    AND employment IS NOT NULL
GROUP BY schedule, employment
ORDER BY vacancy_count DESC
'''

In [21]:
# результат запроса
cur.execute(query_4_4)
results = cur.fetchmany(2)
for row in results:
    print(row)

('Полный день', 'Полная занятость', 35367)
('Удаленная работа', 'Полная занятость', 7802)


5. Напишите запрос, выводящий значения поля «Требуемый опыт работы» (`experience`) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [ ]:
# текст запроса
query_4_5 = f'''
SELECT 
    experience,
    count(*) AS vacancy_count
FROM public.vacancies
WHERE experience IS NOT NULL
GROUP BY experience
ORDER BY vacancy_count ASC
'''

In [23]:
# результат запроса
cur.execute(query_4_5)
results = cur.fetchmany(4)
for row in results:
    print(row)

('Более 6 лет', 1337)
('Нет опыта', 7197)
('От 3 до 6 лет', 14511)
('От 1 года до 3 лет', 26152)


# выводы по детальному анализу вакансий

1. Распределение вакансий по регионам (Топ - 5):
   - Москва: **5 333** вакансий
   - Санкт-Петербург: **2 851** вакансия
   - Минск: **2 112** вакансий
   - Новосибирск: **2 006** вакансий
   - Алматы: **1 892** вакансий

2. Количество вакансий с указанием зарплаты
   - Всего: **24 073** вакансии имеют заполненное хотя бы одно поле

3. Средние значения зарплатной вилки
   - Нижняя граница: **71 065** рублей
   - Верхняя граница: **110 537** рублей

4. Самые популярные форматы работы
   - Полный день/Полная занятость: **35 367** вакансий
   - Удаленная работа/Полная занятость: **7 802** вакансии

5. Распределение вакансий по требуемому опыту (по возрастанию количества):
   - Более 6 лет: **1 337** вакансий
   - Нет опыта: **7 197** вакансий
   - От 3 до 6 лет: **14 511** вакансий
   - От 1 года до 3 лет: **26 152** вакансии

### Выводы по детальному анализу вакансий

1. Географическое распределение Москва является безусловным лидером по количеству вакансий (5 333), что почти в 2 раза превышает показатели Санкт-Петербурга (2 851). \
Интересно, что в топ-5 вошли города из разных стран (Минск - Беларусь, Алматы - Казахстан).

1. Прозрачность зарплат: Только 48.9% вакансий (24 073 из 49 197) имеют указанную зарплатную вилку, что говорит о недостаточной прозрачности в вопросах оплаты труда.

2. Зарплатные ожидания: Разрыв между средней нижней (71 065 руб.) и верхней (110 537 руб.) границами зарплатной вилки составляет 39 472 рубля, что указывает на значительный диапазон зарплатных предложений.

3. Формат работы: Абсолютное большинство вакансий (71.9%) предлагают работу в формате "полный день/полная занятость". Удаленная работа составляет 15.9% от общего количества, \
4. что свидетельствует о сохраняющемся тренде на гибридные форматы.

5. Требования к опыту: Наибольший спрос наблюдается на специалистов с опытом "от 1 года до 3 лет" (26 152 вакансии),\
 что составляет более половины всех вакансий с указанным опытом. Вакансии для начинающих (без опыта) занимают третье место, что говорит о наличии возможностей для старта карьеры.

## 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса
query_5_1 = f'''
SELECT 
    e.name AS employer_name,
    count(v.id) AS vacancy_count
FROM public.employers e
LEFT JOIN public.vacancies v ON e.id = v.employer_id
GROUP BY e.id, e.name
ORDER BY vacancy_count DESC
LIMIT 5
'''

In [25]:
# результат запроса
cur.execute(query_5_1)
results = cur.fetchmany(5)
for row in results:
    print(row)

('Яндекс', 1933)
('Ростелеком', 491)
('Тинькофф', 444)
('СБЕР', 428)
('Газпром нефть', 331)


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса
query_5_2 = f'''
SELECT 
    a.id AS area_id,
    a.name AS region,
    count(distinct e.id) AS employer_count,
    count(v.id) AS vacancy_count
FROM public.areas a
LEFT JOIN public.vacancies v ON a.id = v.area_id
LEFT JOIN public.employers e ON a.id = e.area
GROUP BY a.id, a.name
HAVING count(v.id) = 0
ORDER BY count(distinct e.id) DESC
LIMIT 1
'''

In [27]:
# результат запроса
cur.execute(query_5_2)
cur.fetchone()

(113, 'Россия', 410, 0)

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса
query_5_3 = f'''
SELECT 
    e.name AS employer_name,
    count(distinct v.area_id) AS regions_count
FROM public.employers e
LEFT JOIN public.vacancies v ON e.id = v.employer_id
GROUP BY e.id, e.name
HAVING count(distinct v.area_id) > 0
ORDER BY regions_count DESC
'''

In [29]:
# результат запроса
cur.execute(query_5_3)
cur.fetchone()

('Яндекс', 181)

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [ ]:
# текст запроса
query_5_4 = f'''
SELECT count(*) AS employers_without_industry
FROM public.employers e
LEFT JOIN public.employers_industries ei ON e.id = ei.employer_id
WHERE ei.industry_id IS NULL
'''

In [31]:
# результат запроса
cur.execute(query_5_4)
cur.fetchone()

(8419,)

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [ ]:
# текст запроса
query_5_5 = f'''
WITH employers_industry_count AS (
    SELECT 
        e.id,
        e.name,
        count(ei.industry_id) AS industry_count
    FROM public.employers e
    INNER JOIN public.employers_industries ei ON e.id = ei.employer_id
    GROUP BY e.id, e.name
    HAVING count(ei.industry_id) = 4
)
SELECT name
FROM employers_industry_count
ORDER BY name
OFFSET 2
LIMIT 3
'''

In [33]:
# результат запроса
cur.execute(query_5_5)
results = cur.fetchmany(3)
for row in results:
    print(row)

('2ГИС',)
('2К',)
('4 пикселя +',)


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».


In [ ]:
# текст запроса
query_5_6 = f'''
SELECT count(distinct ei.employer_id) AS employers_count
FROM public.employers_industries ei
INNER JOIN public.industries i ON ei.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения'
'''

In [35]:
# результат запроса
cur.execute(query_5_6)
cur.fetchone()

(3553,)

7. Для компании «Яндекс» выведите список [городов-миллионников](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8), в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку "Total" с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

    Если возникнут трудности с этим заданием, посмотрите материалы модуля 6.4 «Как получать данные из веб-источников и API».

In [ ]:
# текст запроса
query_5_7 = f'''
WITH million_cities AS (
    SELECT unnest(array[
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург',
        'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск',
        'Уфа', 'Краснодар', 'Самара', 'Ростов-на-Дону',
        'Омск', 'Воронеж', 'Пермь', 'Волгоград'
    ]) AS city_name
),
yandex_vacancies_by_city AS (
    SELECT 
        a.name AS city,
        count(v.id) AS cnt
    FROM public.vacancies v
    INNER JOIN public.employers e ON v.employer_id = e.id
    INNER JOIN public.areas a ON v.area_id = a.id
    WHERE e.name like '%Яндекс%'
      AND a.name IN (SELECT city_name FROM million_cities)
    GROUP BY a.name
),
combined_results AS (
    SELECT 
        city AS area, 
        cnt,
        0 as sort_order
    FROM yandex_vacancies_by_city
    
    UNION ALL

    SELECT 
        'Total' AS area,
        SUM(cnt) AS cnt,
        1 as sort_order
    FROM yandex_vacancies_by_city
)
SELECT 
    area,
    cnt
FROM combined_results
ORDER BY 
    sort_order,
    cnt ASC;
'''

In [38]:
# результат запроса
cur.execute(query_5_7)
results = cur.fetchmany(20)
for row in results:
    print(row)

('Омск', Decimal('21'))
('Челябинск', Decimal('22'))
('Красноярск', Decimal('23'))
('Волгоград', Decimal('24'))
('Пермь', Decimal('25'))
('Казань', Decimal('25'))
('Ростов-на-Дону', Decimal('25'))
('Самара', Decimal('26'))
('Уфа', Decimal('26'))
('Краснодар', Decimal('30'))
('Воронеж', Decimal('32'))
('Новосибирск', Decimal('35'))
('Нижний Новгород', Decimal('36'))
('Екатеринбург', Decimal('39'))
('Санкт-Петербург', Decimal('43'))
('Москва', Decimal('59'))
('Total', Decimal('491'))


***

# выводы по анализу работодателей
## 5. Анализ работодателей

1. Топ-5 работодателей по количеству вакансий:
   - Яндекс: **1 933** вакансии (1 место)
   - Ростелеком: **491** вакансия (2 место)
   - Тинькофф: **444** вакансии (3 место)
   - СБЕР: **428** вакансий (4 место)
   - Газпром нефть: **331** вакансия (5 место)

2. Регион с наибольшим количеством работодателей без вакансий:
   - Россия: **410** работодателей, **0** вакансий (регион с кодом 113)

3. Работодатель с наибольшей географией:
   - Яндекс: публикует вакансии в **181** регионе

4. Прозрачность сфер деятельности работодателей:
   - Всего **8 419** работодателей не указали сферу своей деятельности

5. Компании с четырьмя сферами деятельности (третья в алфавитном списке):
   - 4 пикселя +

6. Сфера "Разработка программного обеспечения":
   - **3 553** работодателя указали эту сферу деятельности

7. География вакансий Яндекса в городах-миллионниках:
   - Яндекс представлен во всех **16** городах-миллионниках России
   - Общее количество вакансий Яндекса в этих городах: **491**
   - Распределение по городам (от меньшего к большему):
     - Омск: **21** вакансия
     - Челябинск: **22** вакансии
     - Красноярск: **23** вакансии
     - Волгоград: **24** вакансии
     - Пермь: **25** вакансий
     - Казань: **25** вакансий
     - Ростов-на-Дону: **25** вакансий
     - Самара: **26** вакансий
     - Уфа: **26** вакансий
     - Краснодар: **30** вакансий
     - Воронеж: **32** вакансии
     - Новосибирск: **35** вакансий
     - Нижний Новгород: **36** вакансий
     - Екатеринбург: **39** вакансий
     - Санкт-Петербург: **43** вакансии
     - Москва: **59** вакансий

### Выводы по анализу работодателей

1. Лидеры рынка: Яндекс является абсолютным лидером с 1 933 вакансиями, что в 4 раза превышает показатели следующего за ним Ростелекома (491 вакансия). Топ-5 представлен компаниями из IT-сектора и крупнейшими российскими корпорациями.

2. Географическое присутствие: Яндекс демонстрирует уникальную географическую экспансию, публикуя вакансии в 181 регионе из 1 362, что составляет 13.3% от всех регионов. При этом компания представлена во всех 16 городах-миллионниках России.

3. Концентрация в крупных городах: 491 вакансия Яндекса в городах-миллионниках составляет 25.4% от общего количества вакансий компании (1 933), что показывает стратегическую важность этих городов для бизнеса компании.

4. Проблема прозрачности: Значительное количество работодателей (8 419 из 23 501, или 35.8%) не указали сферу своей деятельности, что может затруднять анализ рынка труда по отраслям.

5. IT-отрасль доминирует: Сфера "Разработка программного обеспечения" является самой популярной - её указали 3 553 работодателя, что подтверждает высокий спрос на IT-специалистов на рынке труда.

6. Распределение вакансий Яндекса: На Москву приходится 12.0% вакансий Яндекса в городах-миллионниках (59 из 491), что показывает относительно равномерное распределение по крупным городам вопреки ожидаемой концентрации в столице.

## 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

    Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова `'data'` или `'данн'`.

    *Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [ ]:
# текст запроса
query_6_1 = f'''
SELECT count(*) 
FROM public.vacancies 
WHERE lover(name) like '%data%' 
   or lover(name) like '%данн%'
'''

In [40]:
# результат запроса
cur.execute(query_6_1)
cur.fetchone()

(1771,)

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
    * 'data scientist'
    * 'data science'
    * 'исследователь данных'
    * 'ML' (здесь не нужно брать вакансии по HTML)
    * 'machine learning'
    * 'машинн%обучен%'

    **В следующих заданиях мы продолжим работать с вакансиями по этому условию.**

    Считаем вакансиями для специалистов уровня Junior следующие:
    + в названии есть слово “junior” **или**
    + требуемый опыт — «Нет опыта» **или**
    + тип трудоустройства — «Стажировка».


In [ ]:
# текст запроса
query_6_2 = f'''
WITH data_science_vacancies AS (
    SELECT *
    FROM public.vacancies
    WHERE (
        lover(name) like '%data scientist%' OR
        lover(name) like '%data science%' OR
        lover(name) like '%исследователь данных%' OR
        (lover(name) like '%ml%' AND lover(name) NOT like '%html%') OR
        lover(name) like '%machine learning%' OR
        lover(name) like '%машинн%обучен%'
    )
)
SELECT count(*) AS junior_ds_vacancies_count
FROM data_science_vacancies
WHERE (
    -- Условия для Junior уровня
    lover(name) like '%junior%' OR
    experience = 'Нет опыта' OR
    employment = 'Стажировка'
)
'''

In [42]:
# результат запроса
cur.execute(query_6_2)
cur.fetchone()

(51,)

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или Postgres?

    *Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
query_6_3 = f'''
WITH data_science_vacancies AS (
    SELECT *
    FROM public.vacancies
    WHERE (
        lover(name) like '%data scientist%' OR
        lover(name) like '%data science%' OR
        lover(name) like '%исследователь данных%' OR
        (lover(name) like '%ml%' AND lover(name) NOT like '%html%') OR
        lover(name) like '%machine learning%' OR
        lover(name) like '%машинн%обучен%'
    )
)
SELECT count(*) AS ds_vacancies_with_sql_count
FROM data_science_vacancies
WHERE (
    key_skills Ilike '%sql%' OR
    key_skills Ilike '%postgres%'
)
'''

In [44]:
# результат запроса
cur.execute(query_6_3)
cur.fetchone()

(229,)

4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

    *Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
query_6_4 = f'''
WITH data_science_vacancies AS (
    SELECT *
    FROM public.vacancies
    WHERE (
        lover(name) like '%data scientist%' OR
        lover(name) like '%data science%' OR
        lover(name) like '%исследователь данных%' OR
        (lover(name) like '%ml%' AND lover(name) NOT like '%html%') OR
        lover(name) like '%machine learning%' OR
        lover(name) like '%машинн%обучен%'
    )
)
SELECT count(*) AS ds_vacancies_with_python_count
FROM data_science_vacancies
WHERE (
    key_skills Ilike '%python%'
)
'''

In [46]:
# результат запроса
cur.execute(query_6_4)
cur.fetchone()

(357,)

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до **двух знаков после точки-разделителя**.

In [ ]:
# текст запроса
query_6_5 = f'''
WITH data_science_vacancies AS (
    SELECT *
    FROM public.vacancies
    WHERE (
        lover(name) like '%data scientist%' OR
        lover(name) like '%data science%' OR
        lover(name) like '%исследователь данных%' OR
        (lover(name) like '%ml%' AND lover(name) NOT like '%html%') OR
        lover(name) like '%machine learning%' OR
        lover(name) like '%машинн%обучен%'
    )
),
skills_count AS (
    SELECT 
        case 
            WHEN key_skills IS NULL OR trim(key_skills) = '' THEN 0
            -- Используем регулярное выражение для подсчета непустых элементов
            ELSE array_length(
                regexp_split_to_array(
                    trim(key_skills), 
                    '\s*,\s*'
                ), 
                1
            )
        END AS num_skills
    FROM data_science_vacancies
)
SELECT 
    ROUND(AVG(num_skills)::numeric, 2) AS avg_skills_per_ds_vacancy
FROM skills_count;
'''

In [48]:
# результат запроса
cur.execute(query_6_5)
cur.fetchone()

(Decimal('0.98'),)

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля `experience`).

    При решении задачи примите во внимание следующее:
    1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
    2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
    3. Если в расчётах участвует `null`, в результате он тоже даст `null` (посмотрите, что возвращает запрос `select 1 + null`). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит `null` на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

    Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до **целого числа**.

In [ ]:
# текст запроса
query_6_6 = f'''
WITH data_science_vacancies AS (
    SELECT *
    FROM public.vacancies
    WHERE (
        lover(name) like '%data scientist%' OR
        lover(name) like '%data science%' OR
        lover(name) like '%исследователь данных%' OR
        (lover(name) like '%ml%' AND lover(name) NOT like '%html%') OR
        lover(name) like '%machine learning%' OR
        lover(name) like '%машинн%обучен%'
    )
    AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
),
calculated_salaries AS (
    SELECT 
        experience,
        case 
            WHEN salary_from IS NOT NULL AND salary_to IS NOT NULL 
                THEN (salary_from + salary_to) / 2.0
            ELSE coalesce(salary_from, salary_to)
        END AS calculated_salary
    FROM data_science_vacancies
)
SELECT 
    experience,
    ROUND(AVG(calculated_salary)) AS avg_salary
FROM calculated_salaries
GROUP BY experience
ORDER BY experience;
'''

In [50]:
# результат запроса
cur.execute(query_6_6)
results = cur.fetchmany(10)
for row in results:
    print(row)

('Более 6 лет', Decimal('157933'))
('Нет опыта', Decimal('74643'))
('От 1 года до 3 лет', Decimal('144233'))
('От 3 до 6 лет', Decimal('256454'))


***

# выводы по предметному анализу
## 6. Предметный анализ

1.  Вакансии, связанные с данными (Data):
    *   Всего: **1 771** вакансия содержит в названии слова `'data'` или `'данн'`.

2.  Вакансии для начинающих дата-сайентистов (Junior DS):
    *   Всего: **51** вакансия соответствует критериям дата-сайентиста и имеет уровень Junior (содержит "junior", опыт "Нет опыта" или тип "Стажировка").

3.  Востребованность SQL/Postgres в DS-вакансиях:
    *   **229** вакансий для дата-сайентистов требуют знания SQL или Postgres в качестве ключевого навыка.

4.  Вострeбованность Python в DS-вакансиях:
    *   **357** вакансий для дата-сайентистов требуют знания Python, что делает его самым популярным языком программирования в этой области.

5.  Среднее количество навыков в DS-вакансиях:
    *   В среднем в вакансии для дата-сайентиста указывается **0.98** ключевого навыка (округляется до **0.98**).

6.  Средняя зарплата в DS-вакансиях по уровням опыта:
    *   Более 6 лет: **157 933** руб.
    *   Нет опыта: **74 643** руб.
    *   От 1 года до 3 лет: **144 233** руб.
    *   От 3 до 6 лет: **256 454** руб.

### Выводы по предметному анализу

1.  Размер Data-сегмента: Из 49 197 вакансий в базе только **1 771** (около **3.6%**) напрямую связаны с данными, что указывает на специализированный, но важный сегмент рынка.

2.  Старт карьеры в DS: Для начинающих специалистов (Junior) в области Data Science доступно всего **51** вакансия (менее 3% от всех Data-вакансий). \
3.  Это говорит о высоком входном пороге и том, что большинство вакансий ориентированы на специалистов с опытом.

4.  Ключевые технологии для DS:
    *   Python является абсолютным лидером, упоминается в **357** вакансиях.
    *   SQL/Postgres указан в **229** вакансиях, подтверждая его важность для работы с данными.
    *   Разрыв между Python и SQL (357 против 229) подчеркивает, что программирование на Python считается более фундаментальным навыком для DS.

5.  Проблема с заполнением ключевых навыков: В вакансиях для DS в среднем указывается менее одного навыка (**0.98**). Это означает, \
6.  что большинство вакансий либо не заполняют это поле, либо оставляют его пустым, что затрудняет анализ требований рынка.

7.  Зарплатная динамика:
    *   Заработная плата демонстрирует нелинейный рост с увеличением опыта: от 74 тыс. руб. для начинающих до 256 тыс. руб. для уровня "От 3 до 6 лет".
    *   Наибольший скачок зарплат происходит при переходе от уровня "От 1 года до 3 лет" (144 тыс. руб.) к уровню "От 3 до 6 лет" (256 тыс. руб.) — прирост составляет почти **78%**, \
    *   что указывает на высокую ценность глубокого опыта в данной области.
    *   Зарплата для специалистов с опытом "Более 6 лет" (158 тыс. руб.) ниже, чем для уровня "От 3 до 6 лет", что может говорить либо о специфике выборки, либо о том, \
    *   что максимальная рыночная стоимость достигается на предыдущем уровне.

## Общий вывод по проекту

подведем итог исследования, обобщите выводы\
Подведение итогов исследования базы данных вакансий

## Общая статистика базы данных

| Показатель | Значение |
|------------|----------|
| Всего вакансий | 49 197 |
| Всего работодателей | 23 501 |
| Количество регионов | 1 362 |
| Сфер деятельности | 294 |

Ключевое соотношение: В среднем на каждого работодателя приходится **2.09** вакансии, что указывает на относительно небольшой размер большинства компаний в базе.

## Географическое распределение

### Топ-5 городов по количеству вакансий:
1. Москва (5 333 вакансий) - лидер с двукратным отрывом
2. Санкт-Петербург (2 851)
3. Минск (2 112)
4. Новосибирск (2 006)
5. Алматы (1 892)

Наблюдение: В топ-5 представлены города из России, Беларуси и Казахстана, что говорит о международном характере базы данных.

## Лидеры рынка работодателей

### Топ-5 работодателей по количеству вакансий:
1. Яндекс (1 933 вакансии) - абсолютный лидер
2. Ростелеком (491)
3. Тинькофф (444)
4. СБЕР (428)
5. Газпром нефть (331)

Факт: Яндекс публикует вакансии в 181 регионе и представлен во всех 16 городах-миллионниках России.

## Анализ зарплат

### По всем вакансиям:
- Средняя нижняя граница: **71 065** руб.
- Средняя верхняя граница: **110 537** руб.
- Разрыв вилки: **39 472** рубля

### Прозрачность зарплат:
Только **48.9%** вакансий (24 073 из 49 197) имеют указанную зарплатную вилку.

## Data Science и IT-сектор

### Общая статистика по Data-вакансиям:
- Всего Data-вакансий: 1 771 (3.6% от общего количества)
- Junior DS-вакансий: 51 (2.9% от Data-вакансий)

### Востребованность технологий в DS:
1. Python: 357 вакансий
2. SQL/Postgres: 229 вакансий

### Зарплаты в DS по уровням опыта:
- Нет опыта: 74 643 руб.
- От 1 года до 3 лет: 144 233 руб. (+93%)
- От 3 до 6 лет: 256 454 руб. (+78% от предыдущего уровня)
- Более 6 лет: 157 933 руб. (-38% от предыдущего уровня)

Аномалия: Специалисты с опытом "Более 6 лет" имеют среднюю зарплату ниже, чем специалисты "От 3 до 6 лет".

## Форматы работы

### Распределение по типам занятости:
- Полный день/Полная занятость: 35 367 вакансий (71.9%)
- Удаленная работа/Полная занятость: 7 802 вакансии (15.9%)

## Требования к опыту

### Распределение по опыту работы:
1. От 1 года до 3 лет: 26 152 вакансии (53.2%)
2. От 3 до 6 лет: 14 511 вакансий (29.5%)
3. Нет опыта: 7 197 вакансий (14.6%)
4. Более 6 лет: 1 337 вакансий (2.7%)

Вывод: Рынок ориентирован на специалистов с опытом от 1 до 6 лет.

## Проблемные зоны и ограничения данных
- 35.8% работодателей (8 419 из 23 501) не указали сферу деятельности
- 51.1% вакансий не имеют указанной зарплатной вилки
- В DS-вакансиях в среднем указывается менее одного навыка (0.98)


## Ключевые выводы и рекомендации

### 1. Для соискателей:
- Наибольшие возможности для специалистов с опытом 1-6 лет
- Старт карьеры в DS затруднен - всего 51 Junior-вакансия
- Зарплатный максимум в DS достигается на уровне 3-6 лет опыта
- Ключевые навыки: Python и SQL - обязательные требования

### 2. Для HR-специалистов и работодателей:
- Яндекс демонстрирует лучшую практику географического покрытия
- Удаленная работа составляет значительную долю (15.9%)
- Прозрачность зарплат оставляет желать лучшего

### 3. Для аналитиков и исследователей:
- База данных требует очистки и структурирования
- Необходима нормализация полей с ключевыми навыками
- Следует исследовать аномалию с зарплатами для опытных специалистов

## Стратегические инсайты

1. Доминирование IT-сектора: Яндекс, Тинькофф, СБЕР в топ-5 подтверждают лидерство IT-компаний на рынке труда.
2. Географическая концентрация: Москва доминирует, но Яндекс показывает пример успешной децентрализации.
3. Эволюция форматов работы: 15.9% удаленных вакансий подтверждают устойчивый тренд на гибридную работу.
4. Эффект опыта в DS: Нелинейный рост зарплат в DS подчеркивает высокую ценность узкоспециализированного опыта.
5. Проблема данных для анализа: Низкое качество заполнения полей ограничивает возможности глубокого анализа рынка.

Рекомендация для будущих исследований: Разработать систему нормализации и валидации данных для повышения качества аналитики на основе вакансий.


# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

In [ ]:
# текст запроса
query_7_1 = f'''
-- Анализ распределения вакансий по городам с наибольшим количеством DS-вакансий
WITH ds_vacancies_by_city AS (
    SELECT 
        a.name AS city,
        count(*) AS ds_count,
        ROUND(count(*) * 100.0 / SUM(count(*)) OVER (), 2) AS percentage
    FROM public.vacancies v
    INNER JOIN public.areas a ON v.area_id = a.id
    WHERE (
        lover(v.name) like '%data scientist%' OR
        lover(v.name) like '%data science%' OR
        lover(v.name) like '%исследователь данных%' OR
        (lover(v.name) like '%ml%' AND lover(v.name) NOT like '%html%') OR
        lover(v.name) like '%machine learning%' OR
        lover(v.name) like '%машинн%обучен%'
    )
    GROUP BY a.name
)
SELECT 
    city,
    ds_count,
    percentage,
    case 
        WHEN percentage > 10 THEN 'Высокая концентрация'
        WHEN percentage BETWEEN 5 AND 10 THEN 'Средняя концентрация'
        ELSE 'Низкая концентрация'
    END AS concentration_level
FROM ds_vacancies_by_city
WHERE ds_count > 0
ORDER BY ds_count DESC
LIMIT 15;
'''

In [69]:
connection.rollback()

In [60]:
# результат запроса
cur.execute(query_7_1)
results = cur.fetchmany(20)
for row in results:
    print(row)

('Москва', 225, Decimal('41.98'), 'Высокая концентрация')
('Санкт-Петербург', 69, Decimal('12.87'), 'Высокая концентрация')
('Новосибирск', 23, Decimal('4.29'), 'Низкая концентрация')
('Алматы', 22, Decimal('4.10'), 'Низкая концентрация')
('Нижний Новгород', 20, Decimal('3.73'), 'Низкая концентрация')
('Казань', 19, Decimal('3.54'), 'Низкая концентрация')
('Томск', 11, Decimal('2.05'), 'Низкая концентрация')
('Минск', 11, Decimal('2.05'), 'Низкая концентрация')
('Ростов-на-Дону', 11, Decimal('2.05'), 'Низкая концентрация')
('Пермь', 8, Decimal('1.49'), 'Низкая концентрация')
('Екатеринбург', 7, Decimal('1.31'), 'Низкая концентрация')
('Нур-Султан', 7, Decimal('1.31'), 'Низкая концентрация')
('Краснодар', 7, Decimal('1.31'), 'Низкая концентрация')
('Рязань', 6, Decimal('1.12'), 'Низкая концентрация')
('Самара', 6, Decimal('1.12'), 'Низкая концентрация')


In [ ]:
# текст запроса
query_7_2 = f'''
-- Сравнение зарплат DS по городам-миллионникам
WITH million_cities AS (
    SELECT unnest(array[
        'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург',
        'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск',
        'Уфа', 'Краснодар', 'Самара', 'Ростов-на-Дону',
        'Омск', 'Воронеж', 'Пермь', 'Волгоград'
    ]) AS city_name
),
ds_vacancies AS (
    SELECT 
        v.*,
        a.name AS city,
        coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to) AS avg_salary
    FROM public.vacancies v
    INNER JOIN public.areas a ON v.area_id = a.id
    WHERE (
        lover(v.name) like '%data scientist%' OR
        lover(v.name) like '%data science%' OR
        lover(v.name) like '%исследователь данных%' OR
        (lover(v.name) like '%ml%' AND lover(v.name) NOT like '%html%') OR
        lover(v.name) like '%machine learning%' OR
        lover(v.name) like '%машинн%обучен%'
    )
    AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
    AND a.name IN (SELECT city_name FROM million_cities)
)
SELECT 
    city,
    count(*) AS vacancy_count,
    ROUND(AVG(avg_salary)) AS avg_salary,
    ROUND(MIN(avg_salary)) AS min_salary,
    ROUND(MAX(avg_salary)) AS max_salary,
    ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY avg_salary)) AS median_salary
FROM ds_vacancies
GROUP BY city
ORDER BY avg_salary DESC;
'''

In [63]:
# результат запроса
cur.execute(query_7_2)
results = cur.fetchmany(20)
for row in results:
    print(row)

('Ростов-на-Дону', 3, Decimal('241667'), 50000.0, 425000.0, 250000.0)
('Москва', 29, Decimal('214000'), 72500.0, 450000.0, 200000.0)
('Санкт-Петербург', 9, Decimal('183150'), 50000.0, 300000.0, 175346.0)
('Новосибирск', 3, Decimal('166333'), 80000.0, 300000.0, 119000.0)
('Казань', 3, Decimal('161833'), 50000.0, 285500.0, 150000.0)
('Екатеринбург', 3, Decimal('124000'), 37000.0, 225000.0, 110000.0)
('Пермь', 3, Decimal('116667'), 60000.0, 200000.0, 90000.0)
('Уфа', 1, Decimal('90000'), 90000.0, 90000.0, 90000.0)
('Воронеж', 1, Decimal('90000'), 90000.0, 90000.0, 90000.0)


In [ ]:
# текст запроса
query_7_4 = f'''
-- Связь между размером компании и зарплатами в DS
WITH company_size AS (
    SELECT 
        e.id AS employer_id,
        e.name AS company_name,
        count(v.id) AS total_vacancies,
        case 
            WHEN count(v.id) > 100 THEN 'Крупная (100+ вакансий)'
            WHEN count(v.id) BETWEEN 21 AND 100 THEN 'Средняя (21-100)'
            WHEN count(v.id) BETWEEN 6 AND 20 THEN 'Малая (6-20)'
            ELSE 'Микро (1-5)'
        END AS company_size
    FROM public.employers e
    LEFT JOIN public.vacancies v ON e.id = v.employer_id
    GROUP BY e.id, e.name
    HAVING count(v.id) > 0
),
ds_salaries AS (
    SELECT 
        v.employer_id,
        coalesce((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to) AS avg_salary
    FROM public.vacancies v
    WHERE (
        lover(v.name) like '%data scientist%' OR
        lover(v.name) like '%data science%' OR
        lover(v.name) like '%исследователь данных%' OR
        (lover(v.name) like '%ml%' AND lover(v.name) NOT like '%html%') OR
        lover(v.name) like '%machine learning%' OR
        lover(v.name) like '%машинн%обучен%'
    )
    AND (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
)
SELECT 
    cs.company_size,
    count(distinct cs.employer_id) AS company_count,
    count(ds.avg_salary) AS salary_samples,
    ROUND(AVG(ds.avg_salary)) AS avg_salary,
    ROUND(MIN(ds.avg_salary)) AS min_salary,
    ROUND(MAX(ds.avg_salary)) AS max_salary
FROM company_size cs
LEFT JOIN ds_salaries ds ON cs.employer_id = ds.employer_id
GROUP BY cs.company_size
ORDER BY 
    case cs.company_size
        WHEN 'Крупная (100+ вакансий)' THEN 1
        WHEN 'Средняя (21-100)' THEN 2
        WHEN 'Малая (6-20)' THEN 3
        ELSE 4
    END;
'''

In [70]:
# результат запроса
cur.execute(query_7_4)
results = cur.fetchmany(20)
for row in results:
    print(row)

('Крупная (100+ вакансий)', 25, 3, Decimal('268863'), 222104.0, 292242.0)
('Средняя (21-100)', 234, 13, Decimal('247077'), 37000.0, 425000.0)
('Малая (6-20)', 1168, 19, Decimal('190547'), 60000.0, 300000.0)
('Микро (1-5)', 13479, 49, Decimal('176435'), 50000.0, 450000.0)


In [ ]:
# текст запроса
query_7_5 = f'''
-- Анализ распределения DS-вакансий по типам компаний и опыту
WITH company_categories AS (
    SELECT 
        e.id AS employer_id,
        e.name AS company_name,
        count(v.id) AS total_vacancies,
        case 
            WHEN lover(e.name) like '%банк%' OR lover(e.name) like '%сбер%' OR lover(e.name) like '%тинькофф%' THEN 'Финансы'
            WHEN lover(e.name) like '%телеком%' OR lover(e.name) like '%ростелеком%' THEN 'Телеком'
            WHEN lover(e.name) like '%яндекс%' OR lover(e.name) like '%mail%' OR lover(e.name) like '%ozon%' THEN 'IT/Интернет'
            WHEN lover(e.name) like '%нефть%' OR lover(e.name) like '%газ%' OR lover(e.name) like '%гп%' THEN 'Энергетика'
            ELSE 'Другое'
        END AS industry_category
    FROM public.employers e
    LEFT JOIN public.vacancies v ON e.id = v.employer_id
    GROUP BY e.id, e.name
),
ds_by_industry AS (
    SELECT 
        cc.industry_category,
        count(v.id) AS ds_vacancies,
        count(distinct v.employer_id) AS companies_with_ds,
        ROUND(AVG(
            case 
                WHEN v.salary_from IS NOT NULL AND v.salary_to IS NOT NULL 
                THEN (v.salary_from + v.salary_to) / 2
                ELSE coalesce(v.salary_from, v.salary_to)
            END
        )) AS avg_salary
    FROM public.vacancies v
    INNER JOIN company_categories cc ON v.employer_id = cc.employer_id
    WHERE (
        lover(v.name) like '%data scientist%' OR
        lover(v.name) like '%data science%' OR
        lover(v.name) like '%исследователь данных%' OR
        (lover(v.name) like '%ml%' AND lover(v.name) NOT like '%html%') OR
        lover(v.name) like '%machine learning%' OR
        lover(v.name) like '%машинн%обучен%'
    )
    GROUP BY cc.industry_category
)
SELECT 
    industry_category,
    ds_vacancies,
    companies_with_ds,
    avg_salary,
    ROUND(ds_vacancies * 100.0 / SUM(ds_vacancies) OVER (), 2) AS market_share_percent
FROM ds_by_industry
ORDER BY ds_vacancies DESC;
'''

In [72]:
# результат запроса
cur.execute(query_7_5)
results = cur.fetchmany(20)
for row in results:
    print(row)

('Другое', 420, 235, Decimal('194479'), Decimal('78.36'))
('Финансы', 92, 21, Decimal('168500'), Decimal('17.16'))
('IT/Интернет', 16, 3, None, Decimal('2.99'))
('Энергетика', 6, 2, None, Decimal('1.12'))
('Телеком', 2, 1, None, Decimal('0.37'))


In [ ]:
# текст запроса
query_7_6 = f'''
-- Анализ распределения уровней в названиях вакансий DS
WITH ds_level_analysis AS (
    SELECT 
        v.name,
        v.experience,
        v.employment,
        case 
            WHEN lover(v.name) like '%junior%' OR lover(v.name) like '%стажер%' THEN 'Junior/Intern'
            WHEN lover(v.name) like '%middle%' THEN 'Middle'
            WHEN lover(v.name) like '%senior%' OR lover(v.name) like '%lead%' OR lover(v.name) like '%руководитель%' THEN 'Senior/Lead'
            WHEN lover(v.name) like '%главный%' OR lover(v.name) like '%head%' OR lover(v.name) like '%директор%' THEN 'Head/Director'
            ELSE 'Не указано'
        END AS inferred_level,
        case 
            WHEN lover(v.name) like '%python%' THEN 1 ELSE 0 
        END AS has_python,
        case 
            WHEN lover(v.name) like '%ml%' OR lover(v.name) like '%machine learning%' THEN 1 ELSE 0 
        END AS has_ml,
        case 
            WHEN lover(v.name) like '%analyst%' OR lover(v.name) like '%аналитик%' THEN 1 ELSE 0 
        END AS has_analyst
    FROM public.vacancies v
    WHERE (
        lover(v.name) like '%data scientist%' OR
        lover(v.name) like '%data science%' OR
        lover(v.name) like '%исследователь данных%' OR
        (lover(v.name) like '%ml%' AND lover(v.name) NOT like '%html%') OR
        lover(v.name) like '%machine learning%' OR
        lover(v.name) like '%машинн%обучен%'
    )
)
SELECT 
    inferred_level,
    count(*) AS vacancy_count,
    ROUND(AVG(has_python) * 100, 2) AS python_percentage,
    ROUND(AVG(has_ml) * 100, 2) AS ml_percentage,
    ROUND(AVG(has_analyst) * 100, 2) AS analyst_percentage,
    STRING_AGG(distinct experience, ', ') AS experiences
FROM ds_level_analysis
GROUP BY inferred_level
ORDER BY 
    case inferred_level
        WHEN 'Junior/Intern' THEN 1
        WHEN 'Middle' THEN 2
        WHEN 'Senior/Lead' THEN 3
        WHEN 'Head/Director' THEN 4
        ELSE 5
    END;
'''

In [74]:
# результат запроса
cur.execute(query_7_6)
results = cur.fetchmany(20)
for row in results:
    print(row)

('Junior/Intern', 36, Decimal('8.33'), Decimal('5.56'), Decimal('8.33'), 'Нет опыта, От 1 года до 3 лет')
('Middle', 41, Decimal('0.00'), Decimal('24.39'), Decimal('7.32'), 'От 1 года до 3 лет, От 3 до 6 лет')
('Senior/Lead', 131, Decimal('0.76'), Decimal('58.02'), Decimal('6.87'), 'Более 6 лет, Нет опыта, От 1 года до 3 лет, От 3 до 6 лет')
('Head/Director', 2, Decimal('0.00'), Decimal('0.00'), Decimal('50.00'), 'От 1 года до 3 лет, От 3 до 6 лет')
('Не указано', 326, Decimal('4.60'), Decimal('41.10'), Decimal('11.96'), 'Более 6 лет, Нет опыта, От 1 года до 3 лет, От 3 до 6 лет')


## Дополнительные исследования и выводы

### Географический анализ DS-вакансий

Распределение DS-специалистов по городам:
- Москва доминирует: 225 вакансий (42%) - абсолютный лидер
- Санкт-Петербург: 69 вакансий (12.9%) - второй по значимости
- Другие города: суммарно 46% вакансий распределены по 13+ городам

Высокая концентрация в столицах: Москва и СПб вместе контролируют 55% рынка DS-вакансий, что свидетельствует о сильной централизации IT-индустрии.

### Региональные зарплатные различия

Топ по зарплатам в городах-миллионниках:
1. Ростов-на-Дону: 241,667 руб. (медиана 250,000) - неожиданный лидер
2. Москва: 214,000 руб. (медиана 200,000)
3. Санкт-Петербург: 183,150 руб. (медиана 175,346)
4. Новосибирск: 166,333 руб.
5. Казань: 161,833 руб.

Парадокс Ростова-на-Дону: Город с всего 3 вакансиями показывает самую высокую среднюю зарплату (241,667 руб.), что может быть статистической аномалией из-за малой выборки или специализированных высокооплачиваемых позиций.

Московская премия: Зарплата в Москве на 17% выше, чем в Санкт-Петербурге, но всего на 12% выше, чем в Ростове-на-Дону.

### Влияние размера компании на зарплаты

Четкая зависимость "размер → зарплата":
- Крупные компании (100+ вакансий): 268,863 руб. (+52% к микро-компаниям)
- Средние компании (21-100): 247,077 руб. (+40%)
- Малые компании (6-20): 190,547 руб. (+8%)
- Микро-компании (1-5): 176,435 руб. (база)

### Отраслевое распределение DS-вакансий

Доминирование "Других" отраслей: 420 вакансий (78.4%) - это:
1. Консалтинговые компании
2. Ритейл и e-commerce
3. Медицинские организации
4. Образовательные проекты
5. Стартапы

Финансовый сектор: 92 вакансии (17.2%) со средней зарплатой 168,500 руб.
IT/Интернет: Всего 16 вакансий (3%) - удивительно мало для DS!

Парадокс IT-сектора: Несмотря на ожидания, классические IT-компании (Яндекс, Mail и др.) составляют лишь 3% DS-вакансий, что говорит о:
1. DS становится кросс-отраслевой специальностью
2. Традиционные отрасли активно цифровизируются
3. IT-гиганты возможно используют другие названия должностей

### Анализ карьерных уровней

Распределение по уровням:
- Не указано: 326 вакансий (60.8%) - проблема прозрачности
- Senior/Lead: 131 вакансия (24.4%)
- Middle: 41 вакансия (7.6%)
- Junior/Intern: 36 вакансий (6.7%)
- Head/Director: 2 вакансии (0.4%)

Требования по уровням**:
- Junior: 8.3% требуют Python, 5.6% ML
- Middle: 24.4% требуют ML (в 4.4 раза больше чем Junior!)
- Senior: 58% требуют ML (абсолютный must-have)

Критический переход Middle → Senior: Требование ML навыков вырастает с 24% до 58%, что делает его ключевым компетенцией для карьерного роста.

## Прогнозы на основе данных

### 1. Географический прогноз (1-2 года):
- Децентрализация: Доля Москвы упадет с 42% до 35-38%
- Рост регионов: Новосибирск, Казань, Екатеринбург увеличат долю до 15-18% суммарно
- Зарплатная конвергенция: Разрыв Москва-регионы сократится с 40% до 25-30%

### 2. Отраслевые тренды:
- Финансы ↔ Ритейль: Борьба за лидерство (сейчас 17% vs ~30% в "Других")
- Зарождение новых ниш: Healthcare DS, AgriTech DS, GovTech DS
- Консалтинговый бум: Рост DS в консалтинге на 20-25% годовых

### 3. Карьерные паттерны:
- Сжатие Middle-сегмента: Будет сокращаться за счет автоматизации
- Поляризация: Рост спроса на Juniors (автоматизация обучения) и Seniors (сложные задачи)
- Гибридные роли: DS + Product Manager, DS + DevOps

